In [36]:
# import all modules required
import numpy as np              # for the arrays  
import matplotlib.pyplot as plt # for the plotting
import scipy.integrate as si    # to integrate numerically the equations of motion
import scipy.optimize as so     # for the bisection algorithm

# change to qt for plots in a new window
%matplotlib qt

### Parameters (spring)

In [121]:
# mass of squash ball
m = 0.050 # kg

# diameter of squash ball
d =  0.040 # mm

# mass of carrier
mc = 0.025 # kg
# mass of ball + carrier
mt = m + mc

# Number of springs in series
Ns = 2

# Number of springs in parallel
Np = 2

# Length of spring
L0 = 0.05 * Ns # m
# Spring constant
k = 1000/Ns*Np # N/m

# Launch angle
theta = 50/180*np.pi # rad

# Gravitational acceleration
g = 9.81 # m/s^2

# Friction factor
f = 0.5 # N/(m/s)

# Spring position
xs = 0.2 # m
ys = 0.1 # m

### Parameters (Trajectory)

In [122]:
# density and dynamic viscosity of air
rho = 1.204    # kg/m^3
mu  = 1.825e-5 # Pa s 

### Initial Conditions for spring

In [123]:
# Initial spring compression
s0 = 0.02 # m

# Initial velocity
v0 = 0.0 # m/s

y0 = [s0, v0]

### Spring equation

In [124]:
# Spring equation
def spring(y, t):
    # unpack
    s, v = y
    # dsdt
    dsdt = -v
    # dvdt
    dvdt = k*s/mt - g*np.sin(theta) - f*v/mt
    
    return dsdt, dvdt

### Find maximum launching velocity

In [125]:
def maxVelocity(t_end):
    t = np.linspace(0, t_end, 100)
    sol = si.odeint(spring, y0, t)
    v, a = spring(sol[-1], t_end)
    return a

In [126]:
plt.clf()
y = []
for s0 in np.array([0.02, 0.023, 0.026, 0.029 ,0.032, 0.035, 0.038, 0.041, 0.044, 0.047, 0.05])*Ns:
    y0 = [s0, v0]

    t_end = so.newton(maxVelocity, 0.013)

    t = np.linspace(0,t_end,100)

    sol = si.odeint(spring, y0, t)

    s = sol[:,0]
    v = sol[:,1]
    y.append(v[-1])
    plt.plot(t, v, label='s0=%0.2f m' % s0)
    
plt.legend()
plt.xlabel('time (s)')
plt.ylabel('Velocity (m/s)')

print(y)

[4.355392161891462, 5.018037952412451, 5.680683743321095, 6.343329534118095, 7.005975324956749, 7.668621114198129, 8.3312669010191, 8.993912685989494, 9.656558469953742, 10.319204262022046, 10.981850047277145]


### Define functions/equations that needed for trajectory

In [127]:
# obtain drag coefficient from Reynolds number
def get_CD(Re):
    return 24/Re * (1 + 0.27*Re)**0.43 + 0.47 * (1 - np.exp(-0.04*Re**0.38))

In [128]:
# compute right hand side of governing equations at a generic
# position u, defining the x, y, Vx and Vy state variables
def squashball(u, t):
    # unpack u
    x, y, Vx, Vy = u
    
    # find theta
    theta = np.arctan2(Vy, Vx)
    
    # magnitude of velocity
    V_mag = np.sqrt(Vx**2 + Vy**2)
    
    # compute Reynolds number
    Re = rho * V_mag * d / mu
    
    # drag coefficient
    CD = get_CD(Re)
    
    # calculate drag (AND HERE IS WHERE THE MISTAKE WAS!!!)
    D_mag = 0.5 * rho * V_mag**2 * CD * d**2
    
    # calculate drag components
    Dx = D_mag * np.cos(theta)
    Dy = D_mag * np.sin(theta)
    
    # calculate acceleration
    ax = - Dx/m
    ay = - Dy/m - g
    
    # return velocity and acceleration
    return [Vx, Vy, ax, ay]

### Trajectory 

In [129]:
# return t, x, y, Vx, Vy given final integral time T
def trajectory(x0, y0, Vx0, Vy0, T=1):
    # pack IC into u0
    u0 = [x0, y0, Vx0, Vy0]
    
    # define array of times
    t  = np.linspace(0, T, 101)
    
    # integrate forward
    out = si.odeint(squashball, u0, t)

    # unpack data
    x  = out[:, 0]
    y  = out[:, 1]
    Vx = out[:, 2]
    Vy = out[:, 3]
    
    return t, x, y, Vx, Vy

### Trajectory with stop 

In [130]:
# return t, x, y, Vx, Vy from given IC with y[-1] = 0
def trajectory_with_stop(x0, y0, Vx0, Vy0):

    # return y at t=T from give initial conditions
    def find_y_at_T(T):
        t, x, y, Vx, Vy = trajectory(x0, y0, Vx0, Vy0, T=T)
        return y[-1]
    
    # find T_stop such that y(T_stop) = 0
    # find T_stop such that find_y_at_T(T_stop) = 0
    T_stop = so.newton(find_y_at_T, 2)
    
    return trajectory(x0, y0, Vx0, Vy0, T=T_stop)

### Define functions for calculating distance

In [131]:
# compute velocity components from velocity magnitude and angle
def mag2comp(V_mag, theta):
    return V_mag * np.cos(theta), V_mag * np.sin(theta)

In [132]:
# return distance given initial velocity (at 45 degrees)
def distance(V_mag, theta):
    # find velocity components
    Vx, Vy = mag2comp(V_mag, np.deg2rad(theta))

    # solve forward
    t, x, y, Vx, Vy = trajectory_with_stop(0, 0, Vx, Vy)
    
    # return final distance
    return x[-1]

In [161]:
# initial velocities
V_mags = y
angle = np.linspace(0, 90, 11)

x = []
for v in V_mags:
    for theta in angle:
        c = distance(v, theta)
        x.append(c)

x1 = x[0:11]
x2 = x[11:22]
x3 = x[22:33]
x4 = x[33:44]
x5 = x[44:55]
x6 = x[55:66]
x7 = x[66:77]
x8 = x[77:88]
x9 = x[99:110]
x10 = x[110:121]
xtotal = []
xtotal.append(x1)
xtotal.append(x2)
xtotal.append(x3)
xtotal.append(x4)
xtotal.append(x5)
xtotal.append(x6)
xtotal.append(x7)
xtotal.append(x8)
xtotal.append(x9)
xtotal.append(x10)

In [163]:
print(x1)
print(x2)
len(x10)

[7.902418062052842e-08, 0.5956146233054389, 1.129546472307478, 1.550727421728379, 1.819405607010244, 1.9105379600011552, 1.8158592599148606, 1.5445669642146462, 1.1225972513574727, 0.5905013878821506, 2.3447940305135764e-16]
[9.093877049958299e-08, 0.7897789656345141, 1.4962687744015049, 2.0524494617015985, 2.4064909180677962, 2.525949147368768, 2.40027751397041, 2.0416465886644994, 1.4840653336437817, 0.7807823624380416, 3.102735163977002e-16]


11

In [165]:
plt.clf()
plt.figure(4)

plt.plot(angle, x1, label='v = %0.2f m/s' % V_mags[0])
plt.plot(angle, x2, label='v = %0.2f m/s' % V_mags[1])
plt.plot(angle, x3, label='v = %0.2f m/s' % V_mags[2])
plt.plot(angle, x4, label='v = %0.2f m/s' % V_mags[3])
plt.plot(angle, x5, label='v = %0.2f m/s' % V_mags[4])
plt.plot(angle, x6, label='v = %0.2f m/s' % V_mags[5])
plt.plot(angle, x7, label='v = %0.2f m/s' % V_mags[6])
plt.plot(angle, x8, label='v = %0.2f m/s' % V_mags[7])
plt.plot(angle, x9, label='v = %0.2f m/s' % V_mags[8])
plt.plot(angle, x10, label='v = %0.2f m/s' % V_mags[9])

plt.legend(loc='best')
plt.grid(1)
plt.xlabel("angle [degrees]")
plt.ylabel("distance [m]")

Text(0, 0.5, 'distance [m]')